In [105]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from tqdm import tqdm
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [169]:
fixtures_df = pd.DataFrame()
for year in [2016, 2017, 2018, 2019, 2020]:
    url = f'https://understat.com/league/EPL/{year}'
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    scripts = soup.find_all('script')
    strings = scripts[1].string
    ind_start = strings.index("('")+2
    ind_end = strings.index("')")
    json_data = strings[ind_start:ind_end]
    json_data = json_data.encode('utf8').decode('unicode_escape')
    data = json.loads(json_data)
    fixtures_df = fixtures_df.append(pd.DataFrame(data).sort_values('datetime'))

In [107]:
game_stats = pd.DataFrame()
matches = fixtures_df[fixtures_df.isResult == True].id
for match in tqdm(matches):
    url = f'https://understat.com/match/{match}'
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    scripts = soup.find_all('script')
    strings = scripts[2].string
    ind_start = strings.index("('")+2
    ind_end = strings.index("')")
    json_data = strings[ind_start:ind_end]
    json_data = json_data.encode('utf8').decode('unicode_escape')
    data = json.loads(json_data)
    for team in ['h', 'a']:
        game_stats_part = pd.DataFrame(data[team]).transpose()
        game_stats_part['match_id'] = match
        game_stats = game_stats.append(game_stats_part)

100%|██████████| 1805/1805 [15:31<00:00,  1.94it/s]


In [108]:
game_stats.to_pickle('game_stats_df.pkl')

In [109]:
game_stats = pd.read_pickle('game_stats_df.pkl')

In [110]:
game_stats

,id,goals,own_goals,shots,xG,time,player_id,team_id,position,player,h_a,yellow_card,red_card,roster_in,roster_out,key_passes,assists,xA,xGChain,xGBuildup,positionOrder,match_id
125031,125031,0,0,0,0,90,1697,91,GK,Eldin Jakupovic,h,0,0,0,0,0,0,0,0,0,1,461
125032,125032,0,0,0,0,90,1685,91,DR,Ahmed Elmohamady,h,0,0,0,0,1,0,0.021670209243893623,0.061049532145261765,0.03937932476401329,2,461
125033,125033,0,0,2,0.05041324719786644,90,1686,91,DC,Curtis Davies,h,1,0,0,0,0,0,0,0,0,3,461
125034,125034,0,0,0,0,90,1689,91,DC,Jake Livermore,h,0,0,0,0,0,0,0,0,0,3,461
125035,125035,0,0,0,0,90,1688,91,DL,Andrew Robertson,h,0,0,0,0,0,0,0,0.06231752410531044,0.06231752410531044,4,461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457323,457323,0,0,0,0,64,6424,81,ML,Ben Johnson,a,0,0,457326,0,0,0,0,0,0,10,14711
457324,457324,0,0,1,0.03431173786520958,90,531,81,FW,Michail Antonio,a,0,0,0,0,1,0,0.041240423917770386,0.07555215805768967,0,15,14711
457325,457325,0,0,2,0.09904623031616211,90,1776,81,FW,Jarrod Bowen,a,0,0,0,0,2,0,0.19072416424751282,0.2897703945636749,0,15,14711
457327,457327,0,0,1,0.05147441849112511,26,535,81,Sub,Manuel Lanzini,a,0,0,0,457322,1,0,0.09359919279813766,0.14507360756397247,0.09359919279813766,17,14711


In [170]:
fixtures_df['home_team'] = [row.get('title') for row in fixtures_df['h']]

In [171]:
fixtures_df['away_team'] = [row.get('title') for row in fixtures_df['a']]

In [172]:
fixtures_df['home_xg'] = [row.get('h') for row in fixtures_df['xG']]

In [173]:
fixtures_df['away_xg'] = [row.get('a') for row in fixtures_df['xG']]

In [174]:
fixtures_df['home_goals'] = [row.get('h') for row in fixtures_df['goals']]

In [175]:
fixtures_df['away_goals'] = [row.get('a') for row in fixtures_df['goals']]

In [176]:
fixtures_df.id = fixtures_df.id.astype('int')

In [151]:
game_stats.match_id = game_stats.match_id.astype('int')

In [178]:
fixtures_df = fixtures_df[['id', 'isResult', 'datetime', 'home_team', 'away_team', 'home_xg','away_xg', 'home_goals', 'away_goals']]

In [179]:
home_fixtures = fixtures_df.rename(columns = {'home_team': 'team',
                             'away_team': 'team_opp',
                             'home_xg': 'xg_team',
                             'away_xg': 'xg_opp',
                             'home_goals': 'team_goals',
                             'away_goals': 'goals_opp'})

In [180]:
away_fixtures = fixtures_df.rename(columns = {'away_team': 'team',
                             'home_team': 'team_opp',
                             'away_xg': 'xg_team',
                             'home_xg': 'xg_opp',
                             'away_goals': 'team_goals',
                             'home_goals': 'goals_opp'})

In [181]:
fixtures = pd.concat([home_fixtures, away_fixtures]).sort_values('datetime')

In [185]:
fixtures = fixtures[fixtures.isResult == True]

In [195]:
fixtures.dtypes

id             int64
isResult        bool
datetime      object
team          object
team_opp      object
xg_team       object
xg_opp        object
team_goals    object
goals_opp     object
dtype: object

In [200]:
fixtures.loc[:, ['xg_team', 'xg_opp', 'team_goals', 'goals_opp']] = fixtures[['xg_team', 'xg_opp', 'team_goals', 'goals_opp']].apply(pd.to_numeric)

/Users/andrewpeters/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1717: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)


In [201]:
fixtures['xg_diff'] = fixtures['xg_team'] - fixtures['xg_opp']

<ipython-input-201-c1eaa725a80e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fixtures['xg_diff'] = fixtures['xg_team'] - fixtures['xg_opp']


In [202]:
fixtures['goal_diff'] = fixtures['team_goals'] - fixtures['goals_opp']

<ipython-input-202-a671b1478c36>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fixtures['goal_diff'] = fixtures['team_goals'] - fixtures['goals_opp']


In [213]:
df = game_stats.merge(fixtures[['id', 'datetime']], left_on = 'match_id', right_on = 'id', how = 'left')

In [214]:
df.player = df.player.str.replace(' ', '_')

In [215]:
df.datetime = pd.to_datetime(df.datetime)

In [216]:
df.to_pickle('match_level_player_stats.pkl')

In [219]:
fixtures.datetime = pd.to_datetime(fixtures.datetime)

/Users/andrewpeters/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [220]:
fixtures.to_pickle('fixture_level_stats.pkl')